In [1]:
#This script is used to optimizely seperate cropland grids by f

In [2]:
import numpy as np
import xarray as xr
from scipy.interpolate import griddata
from scipy.stats import ttest_ind

C:\Users\jiang\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#grid 1 DEG
y1d=np.arange(70,-60.1,-1)
x1d=np.arange(-170,180.1,1)
lon1s,lat1s=np.meshgrid(x1d, y1d)
#2 deg, cropland mask
imai0=np.load('/change to your path/imai0_full.npy',allow_pickle=True)
y3d=np.arange(70,-60.1,-2.)
x3d=np.arange(-170,180.1,2.)
lon3s,lat3s=np.meshgrid(x3d, y3d)

imai_2degf=np.zeros((66,176))*np.nan
for y in range(66):
    for x in range(176):
        al=[imai0[0,y,x],imai0[1,y,x],imai0[2,y,x],imai0[3,y,x],imai0[4,y,x]]
        if np.count_nonzero(np.array(al)>0)>0:
            imai_2degf[y,x]=np.nanmax(al)

In [4]:
#2002-2022 NIRv datasets###############################################################
nirvf=np.load('/change to your path/nir_full_1deg.npy',allow_pickle=True)
nirvf5=np.zeros((21,73,66,176))*np.nan
for yr in range(21):
    for pdt in range(73):
        nirvf5[yr,pdt,:,:] = griddata((lon1s[0:130,0:350].ravel(),lat1s[0:130,0:350].ravel()), nirvf[yr,pdt,::-1,:].ravel(),(lon3s, lat3s) , method='nearest')   
#get rid of non crop & -1 grids
crop_m3=np.where(imai_2degf<2)
nirvf_m5=np.ones((21,73,66,176))*np.nan
for yr in range(21):
    for pdt in range(73):
        mk4=np.where(nirvf5[yr,pdt,:,:]<0)
        i3=nirvf5[yr,pdt,:,:]
        i3[mk4]=np.nan
        i3[crop_m3]=np.nan
        nirvf_m5[yr,pdt,:,:]=i3

In [6]:
#crop calendar data mask
wcu=xr.open_dataset('/change to your path/Wheat.crop.calendar.nc')
loncs,latcs=np.meshgrid(wcu.longitude[19::], wcu.latitude[39:300])#GCE
long,latg=np.meshgrid(wcu.longitude, wcu.latitude)#CCD
imog_nacf=np.load('/change to your path/cecl_gs_0.5_v3.npy',allow_pickle=True)     
#2deg
y1d=np.arange(70,-60.1,-2)
x1d=np.arange(-170,180.1,2)
lon2s,lat2s=np.meshgrid(x1d, y1d)
imog_nac2=np.zeros((2,66,176))*np.nan
imog_nac2[0,:,:] = griddata((loncs.ravel(),latcs.ravel()), imog_nacf[0,:,:].ravel(),(lon2s, lat2s) , method='nearest')   
imog_nac2[1,:,:] = griddata((loncs.ravel(),latcs.ravel()), imog_nacf[1,:,:].ravel(),(lon2s, lat2s) , method='nearest')   

In [7]:
#grow season
nirvf_mean=np.zeros((66,176))*np.nan
for y in range(66):
    for x in range(176):
        sog_na=imog_nac2[0,y,x]
        eog_na=imog_nac2[1,y,x]
        if np.isnan(sog_na)==False and np.isnan(eog_na)==False and imai_2degf[y,x]>=2:
            sogr_na=int(round(sog_na)/5.)#if %5==0, then need to -1
            eogr_na=int(round(eog_na)/5.)
            if sogr_na%5==0:
                sogr_na=sogr_na-1
            if eogr_na%5==0:
                eogr_na=eogr_na-1
            if sogr_na<eogr_na:
                nirl=np.nanmean(nirvf_m5[1:18,sogr_na:eogr_na+1,y,x],axis=0)
                nirvf_mean[y,x]=np.nanmean(nirl)
            if sogr_na>eogr_na:
                nirli=np.append(nirvf_m5[1:18,sogr_na::,y,x],nirvf_m5[2:19,0:eogr_na+1,y,x],axis=1)
                nirl=np.nanmean(nirli,axis=0)
                nirvf_mean[y,x]=np.nanmean(nirl)

C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\2540320667.py:15: RuntimeWarning: Mean of empty slice
  nirl=np.nanmean(nirvf_m5[1:18,sogr_na:eogr_na+1,y,x],axis=0)
C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\2540320667.py:16: RuntimeWarning: Mean of empty slice
  nirvf_mean[y,x]=np.nanmean(nirl)
C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\2540320667.py:19: RuntimeWarning: Mean of empty slice
  nirl=np.nanmean(nirli,axis=0)
C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\2540320667.py:20: RuntimeWarning: Mean of empty slice
  nirvf_mean[y,x]=np.nanmean(nirl)


In [8]:
#beck model based phenology:
#saved phase data
glos_rfs2=np.load('/change to your path/glos_rfs5.npy',allow_pickle=True)#greenup
glos_veg2=np.load('/change to your path/glos_veg5.npy',allow_pickle=True)#end of vegetative
glos_rep2=np.load('/change to your path/glos_rep5.npy',allow_pickle=True)#end of reproductive
glos_rfe2=np.load('/change to your path/glos_rfe5.npy',allow_pickle=True)#harves

In [9]:
#airs f infomation: average during vegetative & reproductive stages
fra_ca5m=np.load('/change to your path/gnip_airs_fra65_A1_new.npy',allow_pickle=True)
fra_vr=np.zeros((66,176))*np.nan
for y in range(66):
    for x in range(176):
        if np.isnan(glos_rfs2[y,x])==False and np.isnan(glos_rep2[y,x])==False:
            d1=int(glos_rfs2[y,x]-1)
            d2=int(glos_rep2[y,x]-1)
            if d1<d2:
                fra_vr[y,x]=np.nanmean(fra_ca5m[d1:d2,y,x])
            if d1>d2:
                fracom4=np.append(fra_ca5m[d1:73,y,x],fra_ca5m[0:d2,y,x])
                fra_vr[y,x]=np.nanmean(fracom4)

In [10]:
fra_vr2=np.zeros((66,176))*np.nan
for y in range(66):
    for x in range(176):
        if np.isnan(fra_vr[y,x])==False:
            fra_vr2[y,x]=fra_vr[y,x]

In [12]:
#read in rainfall data
#2001-2020: mm/day
gpcc_or=np.load('/change to your path/gpcc_full.npy',allow_pickle=True)
longc=np.load('/change to your path/lon_full.npy',allow_pickle=True)
latgc=np.load('/change to your path/lat_full.npy',allow_pickle=True)
lon12g,lat12g=np.meshgrid(longc,latgc)
#interpolated on 73 pentad $ 2deg
#note lat from -89.5
gpcc_it=np.zeros((20,73,180,360))
for yr in range(20):
    for i in range(73):
        if (yr+2001)%4!=0:
            gpcc_it[yr,i,:,:]=np.nanmean(gpcc_or[yr,0+i*5:5+i*5,:,:],axis=0)
        if (yr+2001)%4==0:
            gpcc_new=np.delete(gpcc_or[yr,:,:,:],59,axis=0)
            gpcc_it[yr,i,:,:]=np.nanmean(gpcc_new[0+i*5:5+i*5,:,:],axis=0)
longc2=np.linspace(longc[0],longc[-1],180)
latgc2=np.linspace(latgc[0],latgc[-1],90)
lon22g,lat22g=np.meshgrid(longc2,latgc2)
#check similar 2 different interpolation method
gpcc2deg2=np.zeros((20,73,90,180))*np.nan#2001-2020
for yr in range(20):
    for i in range(73):
        gint=gpcc_it[yr,i,:,:].ravel()
        lons=lon12g.ravel()
        lats=lat12g.ravel()
        gpcc2deg2[yr,i,:,:]=griddata((lons,lats),gint,(lon22g,lat22g),method="nearest")
p_clim=np.nanmean(gpcc2deg2[2:19,:,::-1,:],axis=0)#2003-2019/-90

C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\809522169.py:15: RuntimeWarning: Mean of empty slice
  gpcc_it[yr,i,:,:]=np.nanmean(gpcc_or[yr,0+i*5:5+i*5,:,:],axis=0)
C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\809522169.py:18: RuntimeWarning: Mean of empty slice
  gpcc_it[yr,i,:,:]=np.nanmean(gpcc_new[0+i*5:5+i*5,:,:],axis=0)
C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\809522169.py:30: RuntimeWarning: Mean of empty slice
  p_clim=np.nanmean(gpcc2deg2[2:19,:,::-1,:],axis=0)#2003-2019/-90


In [13]:
#2001-2022 GLEAM PET data
ep_gleam2=np.load('/change to your path/gleam_Ep_2deg3.npy',allow_pickle=True)

In [14]:
#2003-2019
n_clim=np.nanmean(nirvf_m5[1:18,:,:,:],axis=0)
p_clim=np.nanmean(gpcc2deg2[2:19,:,::-1,:],axis=0)#2005-2011/-90
pr_clim=p_clim[:,10:75,5::]
ep_clim=np.nanmean(ep_gleam2[2:19,:,:,:],axis=0)#90

C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\4283930750.py:2: RuntimeWarning: Mean of empty slice
  n_clim=np.nanmean(nirvf_m5[1:18,:,:,:],axis=0)
C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\4283930750.py:3: RuntimeWarning: Mean of empty slice
  p_clim=np.nanmean(gpcc2deg2[2:19,:,::-1,:],axis=0)#2005-2011/-90
C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\4283930750.py:5: RuntimeWarning: Mean of empty slice
  ep_clim=np.nanmean(ep_gleam2[2:19,:,:,:],axis=0)#90


In [15]:
#P & PET during vegetative & reproductive stages
p_vr=np.zeros((65,175))*np.nan
ep_vr=np.zeros((65,175))*np.nan
for y in range(65):
    for x in range(175):
        if np.isnan(glos_rfs2[y,x])==False and np.isnan(glos_veg2[y,x])==False and np.isnan(nirvf_mean[y,x])==False and np.isnan(fra_vr2[y,x])==False:
            d1=int(glos_rfs2[y,x]-1)
            d2=int(glos_rep2[y,x]-1)
            if d1<d2:
                p_vr[y,x]=np.nanmean(pr_clim[d1:d2,y,x])
                ep_vr[y,x]=np.nanmean(ep_clim[d1:d2,y,x])
            if d1>d2:
                pcom2=np.append(pr_clim[d1:73,y,x],pr_clim[0:d2,y,x])
                p_vr[y,x]=np.nanmean(pcom2)
                epcom2=np.append(ep_clim[d1:73,y,x],ep_clim[0:d2,y,x])
                ep_vr[y,x]=np.nanmean(epcom2)

C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\2953417277.py:11: RuntimeWarning: Mean of empty slice
  ep_vr[y,x]=np.nanmean(ep_clim[d1:d2,y,x])
C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\2953417277.py:10: RuntimeWarning: Mean of empty slice
  p_vr[y,x]=np.nanmean(pr_clim[d1:d2,y,x])
C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\2953417277.py:14: RuntimeWarning: Mean of empty slice
  p_vr[y,x]=np.nanmean(pcom2)
C:\Users\jiang\AppData\Local\Temp\ipykernel_18112\2953417277.py:16: RuntimeWarning: Mean of empty slice
  ep_vr[y,x]=np.nanmean(epcom2)


In [16]:
#cumulative P-PET
pec_vr=np.zeros((65,175))*np.nan 
for y in range(65):
    for x in range(175):
        if np.isnan(glos_rfs2[y,x])==False and np.isnan(glos_rep2[y,x])==False and np.isnan(nirvf_mean[y,x])==False and np.isnan(fra_vr2[y,x])==False:
            d1=int(glos_rfs2[y,x]-1)
            d2=int(glos_rep2[y,x]-1)
            if d1<d2:
                pec_vr[y,x]=np.nansum(pr_clim[d1:d2,y,x]-ep_clim[d1:d2,y,x])
            if d1>d2:
                pcom4=np.append(pr_clim[d1:73,y,x],pr_clim[0:d2,y,x])
                epcom4=np.append(ep_clim[d1:73,y,x],ep_clim[0:d2,y,x])
                pec_vr[y,x]=np.nansum(pcom4-epcom4)

In [17]:
pec_vrc=np.zeros((65,175))*np.nan
fra_vrc=np.zeros((65,175))*np.nan
for y in range(65):
    for x in range(175):
        a=[p_vr[y,x],ep_vr[y,x],fra_vr2[y,x]]
        if np.count_nonzero(np.isnan(a))==0:
            pec_vrc[y,x]=pec_vr[y,x]
            fra_vrc[y,x]=fra_vr2[y,x]

In [18]:
#delete NAN values
pec_vric=np.reshape(pec_vrc,65*175)
pec_vric=pec_vric[~np.isnan(pec_vric)]

f_vric=np.reshape(fra_vrc,65*175)
f_vric=f_vric[~np.isnan(f_vric)]

In [20]:
#consider the separation of positive and negative as well
#this aims to find the integer number to optimizely seperate croplands in 2 groups
opa3=np.ones((3,25))*np.nan
gm2=np.ones((2,25))*np.nan
nm2=np.ones((2,25))*np.nan
for k in range(25,50):#find the threshold between 25% - 50%
    pec_rppi=[]
    pec_rpni=[]
    for i in range(int(len(f_vric))):
        if f_vric[i]*100>=k:
            pec_rppi=np.append(pec_rppi,pec_vric[i])#e_rpic = p-pet
        if f_vric[i]*100<k:
            pec_rpni=np.append(pec_rpni,pec_vric[i])
    gm2[0,k-25]=np.nanmean(pec_rpni)
    gm2[1,k-25]=np.nanmean(pec_rppi)
    nm2[0,k-25]=len(pec_rpni)
    nm2[1,k-25]=len(pec_rppi)
    a=ttest_ind(pec_rppi,pec_rpni)
    opa3[0,k-25]=a[0]#difference
    opa3[1,k-25]=a[1]
    opa3[2,k-25]=(np.count_nonzero(pec_rppi<0))*(np.count_nonzero(pec_rpni>0))
    
np.where(opa3[2,:]==np.max(opa3[2,:]))#index of the threshold of f 

(array([10], dtype=int64),)

In [21]:
#find the f threshold using a continuous method (do not constrained to find an integer number)
#prepare for the following bootstrap test method
from scipy.optimize import minimize_scalar 
from scipy.stats import ttest_ind
def optimal_threshold_arrays(f_arr, ppp_arr):
    """
    Determine the optimal f threshold that maximizes the product of:
      - The number of grid cells with positive ∑(P-PET) below the threshold, and
      - The number of grid cells with negative ∑(P-PET) above the threshold.
    
    Additionally, the threshold is only accepted if the two groups (f < threshold and f >= threshold)
    show a statistically significant difference in their cumulative (P-PET) values (p < 0.05, via t-test).
    
    f_arr and ppp_arr are 2D arrays; they are flattened to 1D for the calculation.
    """
    # Flatten the arrays to 1D
    f_flat = f_arr.ravel()
    ppp_flat = ppp_arr.ravel()
    
    def neg_product(thr):
        # Split data into two groups based on the threshold
        group1 = ppp_flat[f_flat < thr]
        group2 = ppp_flat[f_flat >= thr]
        
        # Ensure each group has at least two samples
        if len(group1) < 2 or len(group2) < 2:
            return np.inf
        
        # Perform a two-sample t-test (assuming unequal variances by default)
        t_stat, p_val = ttest_ind(group1, group2, equal_var=False)
        
        # If the two groups are not significantly different, penalize this threshold.
        if p_val > 0.01:
            return np.inf
        
        # Calculate the product of the counts:
        # count of grid cells with positive (P-PET) in group1 and negative (P-PET) in group2.
        count1 = np.sum(group1 > 0)
        count2 = np.sum(group2 < 0)
        product = count1 * count2
        
        # We minimize the negative of the product to maximize the original product.
        return -product
    
    res = minimize_scalar(neg_product, bounds=(f_flat.min(), f_flat.max()), method='bounded')
    return res.x

In [22]:
#bootstrap with random replacemenet to find the 95 percentile window for f threshold
def bootstrap_thresholds_arrays(f_arr, ppp_arr, n_iter=1000):
    """
    Perform bootstrap resampling on the 2D arrays f_arr and ppp_arr to obtain
    a distribution of the optimal f thresholds, then compute the 95% confidence interval.
    
    Parameters:
        f_arr (np.array): 2D array of f values.
        ppp_arr (np.array): 2D array of cumulative (P-PET) values.
        n_iter (int): Number of bootstrap iterations.
    
    Returns:
        boot_thresholds (np.array): Array of optimal thresholds from each bootstrap sample.
        ci (tuple): (ci_lower, ci_upper) representing the 95% confidence interval.
    """
    # Flatten the arrays once for bootstrapping
    f_flat = f_arr.ravel()
    ppp_flat = ppp_arr.ravel()
    N = len(f_flat)
    boot_thresholds = []
    
    for _ in range(n_iter):
        # Resample indices with replacement from the full dataset
        indices = np.random.randint(0, N, size=N)
        f_sample = f_flat[indices]
        ppp_sample = ppp_flat[indices]
        # Calculate the optimal threshold for the bootstrap sample
        thr = optimal_threshold_arrays(f_sample.reshape(-1, 1), ppp_sample.reshape(-1, 1))
        boot_thresholds.append(thr)
    
    boot_thresholds = np.array(boot_thresholds)
    ci_lower = np.percentile(boot_thresholds, 2.5)
    ci_upper = np.percentile(boot_thresholds, 97.5)
    
    return boot_thresholds, (ci_lower, ci_upper)

In [23]:
boot_results, ci = bootstrap_thresholds_arrays(f_vric, pec_vric, n_iter=1000)
print(f"95% confidence interval for f: ({ci[0]:.3f}, {ci[1]:.3f})")

95% confidence interval for f: (0.318, 0.372)


In [24]:
#note: here is the calculation combining AIRS & NIRv, slightly different numbers combining AIRS, TES, NIRv, SIF.
#The 36% f threshold is the average considering different combination 